## Loading Data

In [1]:
import pandas as pd
import datetime
import seaborn as sns
from scipy import stats
import numpy as np
import random

In [2]:
filename = "./../data/311_Cases.csv"

In [4]:
data = pd.read_csv(filename, encoding='utf8',engine='python').dropna()

In [5]:
# quick inspection
len(data) # 3,362,678
data.head()

,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL
24810,3135216,12/05/2013 02:50:24 PM,06/25/2018 09:50:55 AM,06/25/2018 09:50:55 AM,Closed,Found 40 square feet of damaged/uneven sidewal...,DPW BSM Queue,Sidewalk or Curb,Sidewalk_Defect,Collapsed_sidewalk,"920 VAN NESS AVE, SAN FRANCISCO, CA, 94109",6.0,Tenderloin,NORTHERN,37.78418,-122.4208,"(37.78416124, -122.42081673)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...
86201,4203877,11/12/2014 08:46:11 PM,06/25/2018 10:38:19 AM,06/25/2018 10:38:19 AM,Closed,9 Sumner St: damaged sidewalk around utility v...,DPW BSM Queue,Sidewalk or Curb,Sidewalk_Defect,Collapsed_sidewalk,"10 SUMNER ST, SAN FRANCISCO, CA, 94103",6.0,South of Market,SOUTHERN,37.77632,-122.4106,"(37.77629119, -122.41061191)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...
86582,4223752,11/19/2014 10:48:56 AM,06/25/2018 10:22:00 AM,06/25/2018 10:22:00 AM,Closed,2515-2565 Bay Shore Blvd: damaged sidewalk nex...,DPW BSM Queue,Sidewalk or Curb,Sidewalk_Defect,Collapsed_sidewalk,Intersection of BAY SHORE BLVD and SUNNYDALE AVE,10.0,Little Hollywood,INGLESIDE,37.70903,-122.4048,"(37.70885355, -122.4051094)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...
100891,10303151,01/01/2019 10:10:00 PM,01/08/2019 05:46:00 AM,01/08/2019 05:46:00 AM,Closed,01/07/2019 VEH GOA @1033 HOURS #338,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicles,DPT Abandoned Vehicles Low,"1097 OAKDALE AVE, SAN FRANCISCO, CA, 94124",10.0,Hunters Point,BAYVIEW,37.72940,-122.3813,"(37.72939344, -122.38131978)",Web,http://mobile311.sfgov.org/media/san_francisco...
140933,10303744,01/02/2019 08:20:00 AM,01/08/2019 07:39:00 AM,01/08/2019 07:39:00 AM,Closed,01/07/2019 @1349 HRS #168,DPT Abandoned Vehicles Work Queue,Abandoned Vehicle,Abandoned Vehicles,DPT Abandoned Vehicles Low,Intersection of MALTA DR and VALLETTA CT,7.0,Miraloma Park,INGLESIDE,37.73619,-122.4444,"(37.7362185, -122.4444058)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...


## Downsize data

In [6]:
# set random seed
random.seed(42)
small_data = data.sample(n = int(len(data) * 0.1)) # keep 10%
len(small_data) # 336,267

53957

## Select relevant variables
Remember, the goal is to predict how long it takes to close a case, given neighborhood and category of complaint

In [7]:
# note: might want to move Category to Request Type?
X = small_data[["Neighborhood", "Category", ]] # explanatory variables

In [8]:
# NOTE: this takes some time
timedelta = pd.to_datetime(small_data["Closed"]) - pd.to_datetime(small_data["Opened"])
small_data["ClosingTime"] = timedelta.dt.days
Y = small_data["ClosingTime"] # target variable

In [ ]:
len(small_data)

In [9]:
small_data.head()

,CaseID,Opened,Closed,Updated,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL,ClosingTime
2006906,6094995,07/18/2016 05:33:25 PM,08/22/2016 03:57:34 PM,08/22/2016 03:57:34 PM,Closed,cleaned,DPW Ops Queue,Encampments,Encampment Reports,Encampment Cleanup,"470 8TH ST, SAN FRANCISCO, CA, 94103",6.0,South of Market,SOUTHERN,37.77285,-122.4077,"(37.772720378971, -122.407471948151)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...,34
3259051,9112022,06/08/2018 02:17:00 PM,06/11/2018 02:24:00 PM,06/11/2018 02:24:00 PM,Closed,Public Works cleaned up the area. Area cleaned.,HSOC Queue,Encampments,Encampment Reports,Encampment Cleanup,Intersection of POTRERO AVE and 19TH ST,10.0,Mission,MISSION,37.76046,-122.4068,"(37.76066135, -122.4070657)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...,3
2221338,9248800,07/10/2018 02:25:40 PM,07/10/2018 03:25:58 PM,07/10/2018 03:25:58 PM,Closed,Pickup completed.,Recology_Abandoned,Encampments,Encampment items,Cart Pickup,"2230 REVERE AVE, SAN FRANCISCO, CA, 94124",10.0,Apparel City,BAYVIEW,37.73847,-122.4018,"(37.73847079, -122.40189034)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...,0
1691756,5233527,10/28/2015 03:32:48 PM,10/29/2015 07:22:03 AM,10/29/2015 07:22:03 AM,Closed,resolved: PROPERTY HAS ALREADY BEEN POSTED DS....,DPW Ops Queue,Graffiti,Graffiti on Building_residential,Building_residential - Offensive,Intersection of 14TH ST and CASTRO ST,8.0,Corona Heights,PARK,37.76741,-122.4356,"(37.7673279317701, -122.43562427948)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...,0
2761816,7081963,04/25/2017 03:40:10 PM,04/25/2017 06:31:02 PM,04/25/2017 06:31:02 PM,Closed,Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Boxed or Bagged Items,Intersection of GRANT AVE and SACRAMENTO ST,3.0,Chinatown,CENTRAL,37.79337,-122.4060,"(37.79337979, -122.4061295)",Mobile/Open311,http://mobile311.sfgov.org/media/san_francisco...,0


# Outlier Detection: Visual Methods

In [ ]:
sns.distplot(small_data.ClosingTime) # trash

In [ ]:
sns.boxplot(x="ClosingTime", data=small_data) # trash also

In [ ]:
sns.boxplot(x="ClosingTime", y="Category", data=small_data) # not very helpful visually. Too many categories!

In [ ]:
# visualizing the top categories only
small_data.groupby("Category").count().sort_values(by=['CaseID'], ascending=False).head(9) # it was observerd that after 10, the numbers dropped

In [ ]:
top_categories = ["Street and Sidewalk Cleaning", "Graffiti", "Encampments",
                 "Abandoned Vehicle", "Homeless Concerns", "Illegal Postings",
                  "Rec and Park Requests", "Parking Enforcement", "Parking Enforcement",
                  "Sidewalk or Curb"]

top_small_data = small_data[small_data["Category"].isin(top_categories, )]

sns.boxplot(x="ClosingTime", y="Category", data=top_small_data)

# the boxplot is still bad. Thus, we turn to statistical methods

# Outlier Detection 2: Statistical Methods

In [ ]:
z = np.abs(stats.zscore(small_data['ClosingTime'])) # gives us the z-score for each closing time
threshold = 3 # standard deviations

data_reduced = small_data[(z < threshold)] # 
outliers = small_data[(z >= threshold)]

print("Within 3 s.d's       : {}".format(len(data_reduced)))
print("Greather than 3 s.d's: {}".format(len(outliers)))
print("Total                : {}".format(len(small_data)))

# Thus, we've now removed 963 rows of outlier data
# Success!

# Plotting, again

In [ ]:
# After successfully removing outliers, we plot again
sns.distplot(data_reduced.ClosingTime) 

# .. well. We see that there is simply a huge concentration around 
# zero days for closing time. We have to live with this for now.

# Prediction, finally

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# first, let us try without outlier removal

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
clf_2 = KNeighborsClassifier(n_neighbors=5)
clf_2.fit(X_train,y_train)

In [ ]:
from sklearn.metrics import accuracy_score
y_pred_2 = clf_2.predict(X_test)
accuracy_score(y_test,y_pred_2)

# not great. Moving on to add more categorical variables.

# KNN with categorical variables
(still with no outlier removal, yet)

In [ ]:
# note: might want to move Category to Request Type?
# also, we are using 'top small data'
X = top_small_data[["Latitude", "Longitude", "Category"]] # explanatory variables

categories_nums = {"Category": {"Street and Sidewalk Cleaning" : 1,
                                  "Graffiti" :10, 
                                  "Encampments" : 2,
                                  "Abandoned Vehicle": 3, 
                                  "Homeless Concerns": 4, 
                                  "Illegal Postings": 5,
                                  "Rec and Park Requests": 6, 
                                  "Parking Enforcement": 7, 
                                  "Parking Enforcement": 8,
                                  "Sidewalk or Curb": 9}
                  }

top_small_data.replace(categories_nums, inplace = True)
top_small_data.head()

# succesfully replaced

In [ ]:
# nothing in this cell is new, we're just using the updated dataframe
# that contains categorical variables
# train test split, again

X = top_small_data[["Latitude", "Longitude", "Category"]] # explanatory variables
Y = top_small_data["ClosingTime"] # target variable

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33)
clf_2 = KNeighborsClassifier(n_neighbors=5)

clf_2.fit(X_train,y_train)
y_pred_2 = clf_2.predict(X_test)
accuracy_score(y_test,y_pred_2)

# improved!!!! Went to ~50%!

*Suggested next step: add Police District into the model*